This file is the main file where we define the problem data, initiate model and train the model

1. First we import all the relevant classes and libraries

In [2]:
from physicsinformed import InversePhysicsInformedBarModel # py file which has custom class defined
import torch
import math
import matplotlib.pyplot as plt

Custom function needed to generate training and testing data

In [3]:
def generate_grid_1d(length, samples=20, initial_coordinate=0.0):
    """Generate an evenly space grid of a given length and a given number of samples."""

    # Generate the grid
    x = torch.linspace(initial_coordinate, initial_coordinate + length, samples, requires_grad=True)

    # Reshape on a column tensor and return
    return x.view(samples, 1)

2. Analytical 'u' is given

In [4]:
L = 1
x = generate_grid_1d(L, 20)
u_analytic = lambda x: torch.sin(2*math.pi*x)
u = u_analytic(x)

Analytical inverse solution for validation

In [5]:
EA_analytic = lambda x: x**3 - x**2 + 1

3. Problem data are defined. Here the known data like boundary condition and loading conditions are defined

In [6]:
distLoad = lambda x: -2*(3*x**2 - 2*x)*math.pi*torch.cos(2*math.pi*x) + 4*(x**3 - x**2 + 1)*math.pi**2*torch.sin(2*math.pi*x)

4. Next we generate the neural network model using the imported class InversePhysicsInformedBarModel

In [10]:
pinnModel = InversePhysicsInformedBarModel(x, u, L, distLoad) # custom class defined in physicsinformed.py file

5. Next we train our model. The method/function 'train' is defined in the class PhysicsInformedBarModel

In [11]:
epochs = 500
learningRate = 1e-3
weightDecay = 0.1

pinnModel.train(epochs, optimizer='LBFGS', lr=learningRate) # function defined in custom class for training

Epoch: 0/499		Differential equation loss = 20061.244141		Total loss = 20061.244141
Epoch: 100/499		Differential equation loss = 1.147661		Total loss = 1.147661
Epoch: 200/499		Differential equation loss = 0.017053		Total loss = 0.017053
Epoch: 300/499		Differential equation loss = 0.005653		Total loss = 0.005653
Epoch: 400/499		Differential equation loss = 0.004135		Total loss = 0.004135
Epoch: 499/499		Differential equation loss = 0.001258		Total loss = 0.001258


6. We generate sample test data using utilities library and then predict the displacements at those test points

In [13]:
samples = 100
x_test = generate_grid_1d(L, samples)
u_test = u_analytic(x_test)
EA_test = pinnModel.predict(x_test,u_test) # function defined in custom class for predicting EA values

7. We plot predicted coefficient (EA) at test points and also the training history

In [14]:
# plot the results and training losses. Refer the pptx file results slide for more information